In [1]:
import sys
sys.path.insert(0,'../')

In [2]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cv2
from pylab import rcParams
rcParams['figure.figsize'] = 9, 5
# %matplotlib inline
from sklearn.externals import joblib
import windowing.windower as w
import filtering.filter as f
import filtering.history as h
from scipy.ndimage.measurements import label

In [3]:
windows_inited = False
def draw_windows(rgb_image):
    global windows_inited
    global small_windows
    global medium_windows
    global large_windows
    global history
    if not windows_inited:
        small_windows = w.slide_window(img=rgb_image, xy_window=[64, 64], y_start_stop=[400, 496], xy_overlap=[0.85, 0.85])
        medium_windows = w.slide_window(img=rgb_image, xy_window=[96, 96], y_start_stop=[400, 592], xy_overlap=[0.65, 0.65])
        large_windows = w.slide_window(img=rgb_image, xy_window=[128, 128], y_start_stop=[400, 680], xy_overlap=[0.65, 0.65])
        windows_inited = True
    triggered_windows_small = w.search_windows(img=rgb_image, windows=small_windows, classifier=svc, scaler=x_scaler)
    triggered_windows_medium = w.search_windows(img=rgb_image, windows=medium_windows, classifier=svc, scaler=x_scaler)
    triggered_windows_large = w.search_windows(img=rgb_image, windows=large_windows, classifier=svc, scaler=x_scaler)
    
    all_windows = []
    all_windows.extend(triggered_windows_small)
    all_windows.extend(triggered_windows_medium)
    all_windows.extend(triggered_windows_large)

    labels = f.filter_windows(rgb_image=rgb_image, windows=all_windows, history=history)
    image_copy = np.copy(rgb_image)
    if labels[1] > 0:
        f.draw_labeled_bboxes(img=image_copy, labels=labels)
    return image_copy

In [4]:
svc = joblib.load('../saved_models/svc_rgb_all.pkl')
x_scaler = joblib.load('../saved_models/x_scaler_rgb_all.pkl')

In [5]:
history = h.History(n_frames=5)
project_video_name = "project_video.mp4"
dest_video_name = "project_video_avg_heat_maping_5_frames_with_small_windows_.65_to_.85_overlap_pred_conf_thresh=0.65_no_centroid.mp4"
save_file_name = '../output_videos/test/' + dest_video_name
src_video_name = "../test_videos/" + project_video_name
clip1 = VideoFileClip(src_video_name)
srl_clip = clip1.fl_image(draw_windows)
%time srl_clip.write_videofile(save_file_name, audio=False)

[MoviePy] >>>> Building video ../output_videos/test/project_video_avg_heat_maping_5_frames_with_small_windows_.65_to_.85_overlap_pred_conf_thresh=0.65_no_centroid.mp4
[MoviePy] Writing video ../output_videos/test/project_video_avg_heat_maping_5_frames_with_small_windows_.65_to_.85_overlap_pred_conf_thresh=0.65_no_centroid.mp4


100%|█████████▉| 1260/1261 [1:31:12<00:03,  3.56s/it] 


[MoviePy] Done.
[MoviePy] >>>> Video ready: ../output_videos/test/project_video_avg_heat_maping_5_frames_with_small_windows_.65_to_.85_overlap_pred_conf_thresh=0.65_no_centroid.mp4 

CPU times: user 1h 11min 38s, sys: 13.3 s, total: 1h 11min 52s
Wall time: 1h 31min 13s
